<a href="https://colab.research.google.com/github/myPar/NSU_Practice/blob/dev/baseline/LSTM_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Prepare

In [1]:
# imports:
import pandas as pd
import numpy as np
import statistics as st
from google.colab import drive
from pathlib import Path

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

import plotly.express as px
import plotly.graph_objects as go
import sys
import os

In [2]:
drive.mount('/content/drive') # connect to google-drive

dir_name = '/content/drive/MyDrive/Colab Notebooks/'
dataset_file_name = dir_name + 'data.txt'
lstm_models_dir = dir_name + 'LSTM_models/'
gru_models_dir = dir_name + 'GRU_models/'

if dir_name not in sys.path:
  sys.path.append(dir_name)
  
import ml_lib

Mounted at /content/drive


In [3]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/MyDrive/Colab Notebooks/']

In [4]:
data_set_path = Path(dataset_file_name)
df = pd.read_csv(filepath_or_buffer=data_set_path, sep=' ', header=0)

df.head

<bound method NDFrame.head of              Date      Time  qo_lc[m3/d]  qw_lc[m3/d]  qg_lc[m3/d]  \
0     07-May-2014  10:13:26      109.280    -0.312890       3810.3   
1     07-May-2014  10:14:26      113.460    -0.437350       3774.0   
2     07-May-2014  10:15:26      131.630    -0.533550       3892.5   
3     07-May-2014  10:16:26      140.670     0.160910       4043.1   
4     07-May-2014  10:17:26      150.020    -1.145600       4134.6   
...           ...       ...          ...          ...          ...   
1264  08-May-2014  07:34:35       98.744     0.310070       3174.5   
1265  08-May-2014  07:35:35       72.045     0.162760       3134.1   
1266  08-May-2014  07:36:35       87.370    -0.125770       3340.4   
1267  08-May-2014  07:37:35       32.121     0.021522       2834.1   
1268  08-May-2014  07:38:35       35.914     0.228020       2612.4   

      qo_sc[Sm3/d]  qw_sc[Sm3/d]  qg_sc[Sm3/d]  qo_scnp[Sm3/d]  \
0          108.430     -0.314870       92700.0         108.430 

In [5]:
df.columns

Index(['Date', 'Time', 'qo_lc[m3/d]', 'qw_lc[m3/d]', 'qg_lc[m3/d]',
       'qo_sc[Sm3/d]', 'qw_sc[Sm3/d]', 'qg_sc[Sm3/d]', 'qo_scnp[Sm3/d]',
       'qw_scnp[Sm3/d]', 'qg_scnp[Sm3/d]', 'Fo_lc[%]', 'Fw_lc[%]', 'Fg_lc[%]',
       'WLR[%]', 'GVF[%]', 'GLR[m3/m3]', 'BSW[%]', 'GOR[Sm3/Sm3]',
       'GOR1[Sm3/Sm3]', 'mo_lc[kg/d]', 'mw_lc[kg/d]', 'mg_lc[kg/d]',
       'm_lc[kg/d]', 'mo_sc[kg/d]', 'mw_sc[kg/d]', 'mg_sc[kg/d]',
       'Mu_o_lc[cP]', 'Mu_l_lc[cP]', 'Do_lc[g/cm3]', 'Dw_lc[g/cm3]',
       'Dg_lc[g/cm3]', 'Dm_lc[g/cm3]', 'Dl_lc[g/cm3]', 'bo[Sm3/m3]',
       'bw[Sm3/m3]', 'bg[Sm3/m3]', 'Z', 'Rst[Sm3/Sm3]', 'Rwst[Sm3/Sm3]',
       'rgmp[Sm3/Sm3]', 'N32[cps]', 'N81[cps]', 'N356[cps]', 'NTotal[cps]',
       'DeadTime[s]', 'SampleTime[s]', 'DPV[mbar]', 'PL[bara]', 'TL[DegC]',
       'TAMB[DegC]', 'Dyn_DP[mbar]', 'OperatingPointLE[1/m]',
       'OperatingPointHE[1/m]', 'OilPointLE[1/m]', 'OilPointHE[1/m]',
       'WaterPointLE[1/m]', 'WaterPointHE[1/m]', 'GasPointLE[1/m]',
       'GasPoin

In [6]:
selected_rows = ['DPV[mbar]', 'PL[bara]', 'qg_sc[Sm3/d]', 'qo_sc[Sm3/d]', 'TL[DegC]']
outputs = ['qg_sc[Sm3/d]', 'qo_sc[Sm3/d]']
inputs = np.setdiff1d(selected_rows, outputs) # get difference set from two arrays

df = df[selected_rows]

print(df.head)
print("inputs: " + str(inputs))
print("outputs: " + str(outputs))

<bound method NDFrame.head of       DPV[mbar]  PL[bara]  qg_sc[Sm3/d]  qo_sc[Sm3/d]  TL[DegC]
0       1448.70    21.481       92700.0       108.430   -2.5452
1       1462.10    21.473       91895.0       112.580   -2.5323
2       1592.30    21.672       95860.0       130.550   -2.3931
3       1703.30    21.926      100690.0       139.440   -2.0451
4       1821.70    22.196      104290.0       148.640   -1.8009
...         ...       ...           ...           ...       ...
1264    1147.10    23.997       87402.0        97.703   -3.3542
1265    1090.50    23.919       85594.0        71.288   -3.2890
1266    1279.10    24.477       93660.0        86.386   -3.2191
1267     976.62    23.808       76564.0        31.789   -3.2819
1268        NaN       NaN       69609.0        35.567       NaN

[1269 rows x 5 columns]>
inputs: ['DPV[mbar]' 'PL[bara]' 'TL[DegC]']
outputs: ['qg_sc[Sm3/d]', 'qo_sc[Sm3/d]']


### Plot input attributes distribution

In [7]:
for attribute in inputs:
  ml_lib.plot_attibute_destribution(df, attribute)

### Plot output attributes distribution

In [8]:
for attribute in outputs:
  ml_lib.plot_attibute_destribution(df, attribute)

### Create LSTM-net

In [9]:
# model parameters:
input_dim = len(inputs)
output_dim = len(outputs)
neuron_count = 120
timesteps = 60
epoch_count = 500
lr = 0.001
mse = tf.keras.losses.MeanSquaredError()  # loss function

In [10]:
model = tf.keras.Sequential()
model.add(layers.Input((None, input_dim)))
model.add(layers.LSTM(units=neuron_count, activation='tanh', return_sequences=True))
model.add(layers.LSTM(units=output_dim, activation='tanh', return_sequences=True))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 120)         59520     
                                                                 
 lstm_1 (LSTM)               (None, None, 2)           984       
                                                                 
Total params: 60,504
Trainable params: 60,504
Non-trainable params: 0
_________________________________________________________________


### Compile model

In [11]:
model.compile(loss=mse, metrics=[mse], optimizer=Adam(learning_rate=lr))

### Prepare Data

#### Check attributes and supplement data

In [12]:
cardinality_hold = 0.15
expected_col_size = 1269

print("before supplement:")
categorical_attributes = ml_lib.check_attributes_info(df, cardinality_hold, expected_col_size)
print("categorical attributes: " + str(categorical_attributes) + "\n")

ml_lib.supplement_data(df, categorical_attributes)

print("after supplement:")
categorical_attributes = ml_lib.check_attributes_info(df, cardinality_hold, expected_col_size)
print("categorical attributes: " + str(categorical_attributes))

before supplement:
DPV[mbar]: card=1163; skip: percent=0.07880220646178093, count=1
PL[bara]: card=1042; skip: percent=0.07880220646178093, count=1
qg_sc[Sm3/d]: card=1250; skip: percent=0.0, count=0
qo_sc[Sm3/d]: card=1246; skip: percent=0.0, count=0
TL[DegC]: card=1214; skip: percent=0.07880220646178093, count=1
categorical attributes: []

after supplement:
TL[DegC]: card=1215; skip: percent=0.0, count=0
qo_sc[Sm3/d]: card=1246; skip: percent=0.0, count=0
qg_sc[Sm3/d]: card=1250; skip: percent=0.0, count=0
PL[bara]: card=1043; skip: percent=0.0, count=0
DPV[mbar]: card=1164; skip: percent=0.0, count=0
categorical attributes: []


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [13]:
print(df.head)

<bound method NDFrame.head of       TL[DegC]  qo_sc[Sm3/d]  qg_sc[Sm3/d]   PL[bara]    DPV[mbar]
0    -2.545200       108.430       92700.0  21.481000  1448.700000
1    -2.532300       112.580       91895.0  21.473000  1462.100000
2    -2.393100       130.550       95860.0  21.672000  1592.300000
3    -2.045100       139.440      100690.0  21.926000  1703.300000
4    -1.800900       148.640      104290.0  22.196000  1821.700000
...        ...           ...           ...        ...          ...
1264 -3.354200        97.703       87402.0  23.997000  1147.100000
1265 -3.289000        71.288       85594.0  23.919000  1090.500000
1266 -3.219100        86.386       93660.0  24.477000  1279.100000
1267 -3.281900        31.789       76564.0  23.808000   976.620000
1268 -2.607348        35.567       69609.0  22.670506  1176.931364

[1269 rows x 5 columns]>


In [14]:
df.isnull().values.any()

False

#### Normalize dataframe

In [15]:
th_interval = [-1, 1]
sigmoid_interval = [0, 1]

normalized_df = ml_lib.normilize_df(df, th_interval)
print(normalized_df.head)

<bound method NDFrame.head of       TL[DegC]  qo_sc[Sm3/d]  qg_sc[Sm3/d]  PL[bara]  DPV[mbar]
0     0.170802     -0.024995      0.261365 -0.602964   0.156607
1     0.180529      0.022983      0.229137 -0.606450   0.179089
2     0.285499      0.230734      0.387874 -0.519721   0.397529
3     0.547922      0.333511      0.581240 -0.409022   0.583756
4     0.732071      0.439871      0.725364 -0.291349   0.782399
...        ...           ...           ...       ...        ...
1264 -0.439258     -0.149009      0.049262  0.493572  -0.349395
1265 -0.390091     -0.454392     -0.023120  0.459577  -0.444354
1266 -0.337380     -0.279845      0.299798  0.702767  -0.127935
1267 -0.384737     -0.911038     -0.384631  0.411201  -0.635413
1268  0.123936     -0.867361     -0.663070 -0.084547  -0.299346

[1269 rows x 5 columns]>


#### Split data on test and train

In [16]:
df = normalized_df
data = ml_lib.get_train_test(df, 0.3, inputs, outputs)

train_input_data = data.train_input
train_output_data = data.train_output

test_input_data = data.test_input
test_output_data = data.test_output


print("data shapes:")
print(train_input_data.shape)
print(test_input_data.shape)

print(train_output_data.shape)
print(test_output_data.shape)

data shapes:
(888, 3)
(381, 3)
(888, 2)
(381, 2)


### Create tensors

In [17]:
input_train_sequences = ml_lib.get_data_sequence(train_input_data, timesteps, inputs, outputs)
output_train_sequences = ml_lib.get_data_sequence(train_output_data, timesteps, inputs, outputs)

input_test_sequences = ml_lib.get_data_sequence(test_input_data, timesteps, inputs, outputs)
output_test_sequences = ml_lib.get_data_sequence(test_output_data, timesteps, inputs, outputs)

train_sequences_count = len(input_train_sequences)
assert train_sequences_count == len(output_train_sequences)

test_sequences_count = len(input_test_sequences)
assert test_sequences_count == len(output_test_sequences)

print("train batches shapes: ")
ml_lib.print_batches_shapes(input_train_sequences, output_train_sequences)
print()
print("test batches shapes: ")
ml_lib.print_batches_shapes(input_test_sequences, output_test_sequences)

print("\nTrain data:")
print("input batches count=" + str(len(input_train_sequences)))
print("output batches count=" + str(len(output_train_sequences)))

print()

print("Test data:")
print("input batches count=" + str(len(input_test_sequences)))
print("input batches count=" + str(len(output_test_sequences)))

train batches shapes: 
 input batches shapes:
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (48, 3)
 ------------
 output batches shapes:
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (48, 2)

test batches shapes: 
 input batches shapes:
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (21, 3)
 ------------
 output batches shapes:
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (21, 2)

Train data:
input batches count=15
output batches count=15

Test data:
input batches count=7
input batches count=7


### Train model

In [18]:
loss_list = ml_lib.train_model(model, input_train_sequences, output_train_sequences, epoch_count)

epoch-0;{'loss': 0.13189516961574554, 'mean_squared_error': 0.13189516961574554}
epoch-1;{'loss': 0.1252218782901764, 'mean_squared_error': 0.1252218782901764}
epoch-2;{'loss': 0.1217857226729393, 'mean_squared_error': 0.1217857226729393}
epoch-3;{'loss': 0.11881942301988602, 'mean_squared_error': 0.11881942301988602}
epoch-4;{'loss': 0.11537343263626099, 'mean_squared_error': 0.11537343263626099}
epoch-5;{'loss': 0.10957745462656021, 'mean_squared_error': 0.10957745462656021}
epoch-6;{'loss': 0.10281464457511902, 'mean_squared_error': 0.10281464457511902}
epoch-7;{'loss': 0.09777232259511948, 'mean_squared_error': 0.09777232259511948}
epoch-8;{'loss': 0.09262595325708389, 'mean_squared_error': 0.09262595325708389}
epoch-9;{'loss': 0.08770693093538284, 'mean_squared_error': 0.08770693093538284}
epoch-10;{'loss': 0.0832986980676651, 'mean_squared_error': 0.0832986980676651}
epoch-11;{'loss': 0.0791836604475975, 'mean_squared_error': 0.0791836604475975}
epoch-12;{'loss': 0.07548085600137

### Model quality assessment

#### loss

In [19]:
ml_lib.plot_loss(loss_list)

#### Testing model

In [20]:
test_loss = ml_lib.test_model(model, input_test_sequences, output_test_sequences)
print("test loss=" + str(test_loss['loss']))

test loss=0.006916319020092487


#### Making predictions

In [21]:
input_data = df[inputs].to_numpy()
output_data = df[outputs].to_numpy()

print("data shapes:")
print(input_data.shape)
print(output_data.shape)

input_sequences = ml_lib.get_data_sequence(input_data, timesteps, inputs, outputs)
output_sequences = ml_lib.get_data_sequence(output_data, timesteps, inputs, outputs)

seq_count = len(input_sequences)
assert seq_count == len(output_sequences)

print("sequence count=" + str(seq_count))
ml_lib.print_batches_shapes(input_sequences, output_sequences)

predictions = ml_lib.predict(model, input_sequences)

data shapes:
(1269, 3)
(1269, 2)
sequence count=22
 input batches shapes:
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (60, 3)
 (9, 3)
 ------------
 output batches shapes:
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (60, 2)
 (9, 2)


In [22]:
# get attributes predictions from complex tenzors
one_dim_predicted_data = ml_lib.get_1d_data(predictions)

print(one_dim_predicted_data.shape)

predicted_data1 = ml_lib.get_attribute_data(0, one_dim_predicted_data)
predicted_data2 = ml_lib.get_attribute_data(1, one_dim_predicted_data)

print("predicted data shapes: " + str(predicted_data1.shape) + " " + str(predicted_data2.shape))

(1269, 2)
predicted data shapes: (1269,) (1269,)


In [23]:
ml_lib.plot_approximation(outputs[0], df, predicted_data1, 'lines').show()

In [24]:
ml_lib.plot_approximation(outputs[1], df, predicted_data2, 'lines').show()

#### Calculate determination coefficient

In [25]:
actual_data1 = df[outputs[0]].to_numpy()
actual_data2 = df[outputs[1]].to_numpy()

k_1 = ml_lib.get_determination(predicted_data1, actual_data1)
k_2 = ml_lib.get_determination(predicted_data2, actual_data2)

print(outputs[0] + " determination=" + str(k_1))
print(outputs[1] + " determination=" + str(k_2))

qg_sc[Sm3/d] determination=0.9729541692073818
qo_sc[Sm3/d] determination=0.9514930813546937


### Model saving

In [35]:
def get_model_dir(model_type: str):
  if model_type == 'gru':
    model_dir = gru_models_dir
  if model_type == 'lstm':
    model_dir = lstm_models_dir
  else:
    assert False
  
  return model_dir

def get_best_model_loss(model_type: str):
  def get_loss_value(model_name: str):
    metric_string = model_name.partition('_')[2].split('.')[0]  # name_0_1234.h5 -> 0_1234
    metric_string = metric_string.replace('_', '.')

    try:
      metric_value = float(metric_string)
    except ValueError:
      assert False and "invalid model name: " + model_name
    
    return metric_value

  model_dir = get_model_dir(model_type)
  model_names = [item for item in os.listdir(model_dir) if os.path.isfile(item) and item.endwith('.h5')] # get .h5 files from model dir
  
  if len(model_names) == 0:
    return float('inf')
  
  return np.max([get_loss_value(model_name) for model_name in model_names])

def save_model(model, model_loss: float, model_type: str, name_suffix: str):
  def get_model_name():
    string_loss = str(model_loss).replace('.', '_')
    
    return model_type.upper() + name_suffix + string_loss + '.h5'

  best_loss = get_best_model_loss(model_type)

  if model_loss < best_loss:
    model_name = get_model_name()
    model_path = get_model_dir(model_type) + model_name
    model.save(filepath=model_path, save_format='h5')
  else:
    print("current model " + "'" + model_type.upper() + name_suffix + " loss: " + 
          str(model_loss) + " is bigger then minimum loss: " + str(best_loss))




In [36]:
# save lstm+lstm model
model_type = 'lstm'
name_suffix = 'lstm'
save_model(model, loss_list[-1], model_type, name_suffix)

In [37]:
print("model loss=" + str(loss_list[-1]))

model loss=0.006909094285219908
